## Predicting Market Outcomes: Utilizes Data Research and DataFrames to Cluster Data

Utilized Traits:
* Betting Data (Investment Amounts, Frequency, Wallets)
* Outcome Data (Distribution, Winner)
* Market Details (Category, Date, Time Length)

Open DataFrames

In [8]:
import pandas as pd

marketOutcomes = pd.read_csv('data/silver/marketOutcomes.csv')
markets = pd.read_csv('data/silver/markets_with_ai_categories.csv')

Obtain Addresses

In [9]:
addresses = markets['marketMakerAddress']

In [ ]:
import os
from dotenv import load_dotenv
import requests
import pprint
import pandas as pd
load_dotenv()
from datetime import datetime, timedelta
POLYGON_API_KEY = os.getenv("POLYGONSCAN_API_KEY")
print(POLYGON_API_KEY)
def time_decoder(address: str, iter: int = None, sz: str = None, hundredPlusBuys = None, lessThanHundredBuys = None): 
    dictForAddys = {}
    if sz is not None:
        if sz == 'large':
            dictForAddys = hundredPlusBuys[iter]
        elif sz == 'small':
            dictForAddys = lessThanHundredBuys[iter]
    
    else:
        try:
            dictForAddys = pd.read_csv(f'data/bronze/contract_buy_{address}.csv')
        except FileNotFoundError:
            return None
    try:
        people = [address for address in dictForAddys['buyer']]
        people = [addy.lower() for addy in people]
        
    except KeyError:
        return None
    url = (
      f'https://api.polygonscan.com/api'
      f'?module=account'
      f'&action=tokentx'
      f'&address={address}'
      f'&startblock=0'
      f'&endblock=99999999'
      f'&apikey={POLYGON_API_KEY}'
    )
    response = requests.get(url)


    transactionDict = response.json()
    status = transactionDict.get("status")
    if status != '1':
        print(f"Error: {transactionDict.get('message')}")
        return None


    timeDict = {}
    for addy in transactionDict.get('result', []):
      
      if addy.get('from', '').lower() in people:
            try:
               timeDict[addy.get('from','').lower()].append(addy.get('timeStamp'))
            except KeyError:
               timeDict[addy.get('from', '').lower()] = [addy.get('timeStamp')]

    from datetime import datetime

    for addy in timeDict:
        timeDict[addy] = list(map(lambda x: datetime.fromtimestamp(int(x)), timeDict[addy]))
    
    
    allTimes = [ts for ts_list in timeDict.values() for ts in ts_list]

    min_timestamp = min(allTimes)
    
    for addy in timeDict:
        timeDict[addy] = [(ts - min_timestamp) for ts in timeDict[addy]]
    
    dictForAddys['timestamp_difference'] = dictForAddys['buyer'].apply(
        lambda b: timeDict[b][0] if b in timeDict and len(timeDict[b]) > 0 else None
    )
    dictForAddys.to_csv(f'data/bronze/contract_buy_{address}.csv')

    return timeDict

JCBJI6AJS9E71ZVH58E8IT9E8JZBCCRQEE


In [11]:
def serializeTimeDict(timeDict):
    return {
        k: [td.total_seconds() for td in v]
        for k, v in timeDict.items()
    }

In [13]:
timerStamper = []
for index, market in markets.iterrows():
    try:
        buyScans = pd.read_csv(f'data/bronze/contract_buy_{market["marketMakerAddress"]}.csv')
    except FileNotFoundError:
        print(f"Error: 1")
        continue
    try:
        filtered = marketOutcomes.loc[marketOutcomes["marketMakerAddress"] == market["marketMakerAddress"], "index"]
        marketOutcomeIndex = filtered.iloc[0]
    except:
        print(f"Error: 2")
        continue
    
    try:
        buysIndex0 = buyScans[buyScans['outcomeIndex'] == 0]['investmentAmount']
        buysIndex1 = buyScans[buyScans['outcomeIndex'] == 1]['investmentAmount']
    except:
        print(f"Error: 3")
        continue
    try:    
        timeStamp  = time_decoder(market["marketMakerAddress"])
    except:
        print(f"Error: 4")
        continue
    
    try:
        total = sum(buyScans['investmentAmount'].tolist())
        buyScans[buyScans['investmentAmount'] > .05*total]
        buyScansVC = buyScans['outcomeIndex'].value_counts()
        whale0 = buyScansVC[0]
        whale1 = buyScansVC[1]
    except:
        print("Error: 5")
        continue
    
    try: 
        sz = len(buyScans)
        startIndex = int(0.375*sz)
        secondIndex = int(0.625*sz)
        middleRatio = buyScans[startIndex:secondIndex]
        middleRatioVC = middleRatio['outcomeIndex'].value_counts()
        ratio = middleRatioVC.loc[0]/(middleRatioVC.loc[0]+middleRatioVC.loc[1])
    except KeyError:
        print(startIndex)
        print(secondIndex)
        print(ratio)
        print("Error: 6")
        continue
        
    try:
        timerStamper.append({'Market': market['marketMakerAddress'], 'Category': market['category'], 'TimeStamps': serializeTimeDict(timeStamp), 'BuysIndex0': buysIndex0.tolist(), 'BuysIndex1': buysIndex1.tolist(), 'OutcomeIndex': marketOutcomeIndex, 'Whale0': whale0, 'Whale1': whale1, 'MiddleRatio': ratio})
    except:
        print("Timer Failure")
timerStamper = pd.DataFrame(timerStamper)
timerStamper.to_csv('data/silver/timerStamper.csv', index=False)

Error: 1
Error: 1
Error: 1
8
13
0.8846153846153846
Error: 6
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
Error: 1
19
32
0.1
Error: 6
Error: 1
Error: 1
447
745
0.9324324324324325
Error: 6
Error: 1
Error: 1
Error: 1
19
32
0.6956521739130

Creating Official ML Model Training Data

In [ ]:
import pandas as pd
timerStamper = pd.read_csv("data/silver/timerStamper.csv")

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
timerStamper['CategoryEncoded'] = encoder.fit_transform(timerStamper['Category'])
from datetime import timedelta
import ast
dataSector = []
for iter, line in timerStamper.iterrows():
    buysIndex0 = ast.literal_eval(line['BuysIndex0'])
    buysIndex1 = ast.literal_eval(line['BuysIndex1'])
    timeDict = ast.literal_eval(line['TimeStamps'])
    allDeltas = []
    for deltas in timeDict.values():
        allDeltas.extend(deltas)
    maxDelta = max(allDeltas)
    dataSector.append([line['CategoryEncoded'], maxDelta ,len(buysIndex0), sum(buysIndex0), len(buysIndex1), sum(buysIndex1), line['Whale0'], line['Whale1'], line['MiddleRatio']])

Splitting Data into Test and Train

In [15]:
import torch

totalData = len(dataSector)

trainCap = totalData*4 // 5

trainData = dataSector[:trainCap]

testData = dataSector[trainCap:]

trainResults = timerStamper['OutcomeIndex'].tolist()[:trainCap]

testResults = timerStamper['OutcomeIndex'].tolist()[trainCap:]

trainData = torch.tensor(trainData, dtype=torch.float32)
testData = torch.tensor(testData, dtype=torch.float32)
trainResults = torch.tensor(trainResults, dtype=torch.float32)
testResults = torch.tensor(testResults, dtype=torch.float32)



Creating the Model

Model Architecture taken from here : https://medium.com/data-science/pytorch-tabular-binary-classification-a0368da5bb89

Key Point: Relu serves to fid complex relationships between variables
Key Point: Normalizing to keep everything within 0 to 1 scope and not put too much weight on anything
Key Point: Linear Layers Attempt to make reason out of data

In [16]:
import torch
import torch.nn as nn
class MarketPredictor(nn.Module):
    "Initializes multi-class classification model"
    def __init__(self, input_features, output_features, hidden_units=8):
        super().__init__()
        self.layer_1 = nn.Linear(input_features, hidden_units) 
        self.layer_2 = nn.Linear(hidden_units, hidden_units)
        self.layer_out = nn.Linear(hidden_units, output_features) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(hidden_units)
        self.batchnorm2 = nn.BatchNorm1d(hidden_units)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

MarketPredictorModel = MarketPredictor(input_features=9,
                    output_features=1,
                    hidden_units=16)

lossFn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(MarketPredictorModel.parameters(), lr=0.001)



A way to Analyze Accuracy (Literally just a percent)

In [17]:
def binaryAccuracy(actualOutcomes, predProbs, threshold=0.5):
    preds = (predProbs > threshold).float()
    correct = (preds == actualOutcomes).float().sum()
    acc = correct / actualOutcomes.shape[0]
    return acc * 100

Training Data

Both training and testing

In [18]:
def train(epochs: int):
  maxAcc = 0
  for epoch in range(epochs):
    MarketPredictorModel.train()
    binaryPredictions = MarketPredictorModel(trainData).squeeze()
    loss = lossFn(binaryPredictions, trainResults)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    binaryPredictions = torch.sigmoid(binaryPredictions)
    accuracy = binaryAccuracy(trainResults, binaryPredictions)
    MarketPredictorModel.eval()
    
    with torch.inference_mode():
      testPredictions = MarketPredictorModel(testData).squeeze()
      lossTest = lossFn(testPredictions, testResults)
      testPredictions = torch.sigmoid(testPredictions)
      accTest = binaryAccuracy(testResults, testPredictions)
      print(f"Epoch: {epoch} | Loss: {loss:.5f} | Acc: {accuracy:.2f}% | Test Loss: {lossTest:.5f} | Test Acc: {accTest:.4f}")
      maxAcc = max(maxAcc, accTest)
      
      
  return maxAcc

Training Data to be 85% accuracte

In [21]:
maxAcc = 0
while maxAcc < 85:
    maxAcc = train(50)
    print(maxAcc)
    
torch.save(MarketPredictorModel.state_dict(), "MarketPredictor.pt")

Epoch: 0 | Loss: 0.33419 | Acc: 82.92% | Test Loss: 34107848.00000 | Test Acc: 83.6879
Epoch: 1 | Loss: 0.32135 | Acc: 85.05% | Test Loss: 33986028.00000 | Test Acc: 84.3972
Epoch: 2 | Loss: 0.31867 | Acc: 84.34% | Test Loss: 33818368.00000 | Test Acc: 84.3972
Epoch: 3 | Loss: 0.32766 | Acc: 83.45% | Test Loss: 33634696.00000 | Test Acc: 80.1418
Epoch: 4 | Loss: 0.31865 | Acc: 83.63% | Test Loss: 33397556.00000 | Test Acc: 81.5603
Epoch: 5 | Loss: 0.31991 | Acc: 83.99% | Test Loss: 33140940.00000 | Test Acc: 85.1064
Epoch: 6 | Loss: 0.32999 | Acc: 83.99% | Test Loss: 32960846.00000 | Test Acc: 85.1064
Epoch: 7 | Loss: 0.33747 | Acc: 83.63% | Test Loss: 32806262.00000 | Test Acc: 83.6879
Epoch: 8 | Loss: 0.33525 | Acc: 83.63% | Test Loss: 32628804.00000 | Test Acc: 80.8511
Epoch: 9 | Loss: 0.32734 | Acc: 83.81% | Test Loss: 32390054.00000 | Test Acc: 83.6879
Epoch: 10 | Loss: 0.33249 | Acc: 83.81% | Test Loss: 32164740.00000 | Test Acc: 85.8156
Epoch: 11 | Loss: 0.31883 | Acc: 85.23% | 

Analyzing Parameters

In [23]:
for name, param in MarketPredictorModel.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Shape: {param.shape}")
        print(param.data)
        print("-" * 40)

Layer: layer_1.weight | Shape: torch.Size([16, 9])
tensor([[-1.7115e-01, -3.7545e+00, -1.2932e+00,  1.1427e-01,  2.1026e-01,
          1.3453e-01, -1.5897e+00, -4.4725e-01,  1.6826e-01],
        [ 1.5151e-01,  1.6357e+00,  2.5756e+00, -2.6364e-01, -2.4373e+00,
          3.4486e-01,  2.5141e+00, -1.9544e+00,  3.0063e-01],
        [-2.2408e-01,  4.0823e-01, -2.4933e-01,  1.9796e-01,  2.0424e+00,
          2.5781e-01,  2.2168e-01,  1.6536e+00, -1.6025e-03],
        [ 1.3308e-01,  1.1516e-01,  3.0190e-01, -7.6642e-02, -9.5842e-03,
         -3.4621e-02, -2.1722e-02,  2.4485e-01,  3.2722e-01],
        [ 2.5539e-01, -2.7394e-01, -2.4948e-01, -2.5126e-01,  1.8443e-01,
         -1.4166e-01, -2.2776e-01, -1.4142e-01,  9.4897e-02],
        [ 1.2734e-01,  3.6855e+00, -1.7686e+00,  2.0809e-01,  1.9134e+00,
         -8.2400e-02, -1.5280e+00,  2.0358e+00, -2.6768e-01],
        [ 6.1538e-02, -2.4924e-01,  1.0985e+00,  2.8568e-01,  2.5960e-01,
          2.1581e-01,  8.5299e-01, -2.2972e-01,  1.0610e-01